### Merge data with Google Big Query

In [1]:
import pandas as pd

In [2]:
bq = pd.read_csv('data/bq_company.csv')
web = pd.read_csv('data/web_company_data.csv')
patent = pd.read_csv('data/Company_Patent.csv')

In [3]:
patent.dropna(inplace = True)
patent.tail()

,firm_name,fyear,unique_firm_id
213188,"Harward Irrigation Systems, Inc.",1987,213189
213189,Southern Ionics Incorporated,2007,213190
213190,"iMatte, Inc.",2001,213191
213191,Bergman Design Consortium,2004,213192
213192,"LEDS America, Inc.",2014,213193


In [4]:
patent.rename(columns = {'firm_name': 'company_name'}, inplace = True)

### Convert uppercase to lowercase

In [5]:
bq['lower'] = bq['company_name'].apply(lambda x: x.lower())
web['lower'] = web['company_name'].apply(lambda x: x.lower())
patent['lower'] = patent['company_name'].apply(lambda x: x.lower())


### Merge - 1206 company names

In [6]:
merge_company_name = list(bq.merge(patent, on = 'lower', how = 'inner')['lower'])
len(merge_company_name)

143

In [7]:
merge_company_name.extend(list(bq.merge(web, on = 'lower', how = 'inner')['lower']))

In [8]:
merge_company_name = list(set(merge_company_name))
len(merge_company_name)

1851

In [9]:
'facebook' in list(patent['company_name'])

False

In [10]:
'microsoft' in merge_company_name

True

### Get number of repo for merged company
Add tensorflow

In [11]:
merge_company_name.append('tensorflow')

In [12]:
bq_repo = pd.read_csv('data/bq_merge_repo_name.csv')
bq_repo['lower'] = bq_repo['company_name'].apply(lambda x: x.lower())
bq_repo.head()

,repo_name,company_name,lower
0,0--------------------------------------/Trivia...,0--------------------------------------,0--------------------------------------
1,0----0/sauna,0----0,0----0
2,0--key/0--key.github.io,0--key,0--key
3,0--key/lib,0--key,0--key
4,0--key/org-pub,0--key,0--key


In [13]:
merge = pd.DataFrame(data = merge_company_name, 
             columns = ['lower'])
merge.head()

,lower
0,comco
1,stitch
2,juxt
3,squarescale
4,bloomberg


In [14]:
merge_df = bq_repo.merge(merge, on = 'lower', how = 'inner')
merge_df.shape

(16467, 3)

In [16]:
merge_df.iloc[:, :2].to_csv('merge_bq_repo_name.csv', index = False)

In [45]:
merge_df.to_csv('merge_bq_repo_name.csv', index = False, header = None)

In [42]:
merge_df[merge_df['company_name'] == 'airbnb']

,repo_name,company_name,lower
4278,airbnb/AirMapView,airbnb,airbnb
4279,airbnb/Backbone-relational,airbnb,airbnb
4280,airbnb/RxGroups,airbnb,airbnb
4281,airbnb/aerosolve,airbnb,airbnb
4282,airbnb/airflow,airbnb,airbnb
...,...,...,...
4341,airbnb/tapkulibrary,airbnb,airbnb
4342,airbnb/wechat-rails,airbnb,airbnb
4343,airbnb/wraithdb,airbnb,airbnb
4344,airbnb/zendesk_api_client_rb,airbnb,airbnb


In [16]:
'tensorflow' in list(merge_df['lower'])

True

### Fuzzy Merge

In [44]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/Users/ellesmac/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [47]:
fuzz.token_sort_ratio("Catherine Gitau M.", "Gitau Catherine")

94

In [48]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    df_1 is the left table to join
    df_2 is the right table to join
    key1 is the key column of the left table
    key2 is the key column of the right table
    threshold is how close the matches should be to return a match, based on Levenshtein distance
    limit is the amount of matches that will get returned, these are sorted high to low
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

In [ ]:
for i in tqdm(range(1)):
    fuzzy_merge(bq, web, 'company_name', 'company_name', threshold=80)
